In [1]:
import pandas as pd
import numpy as np

In [2]:
# 设定工作目录，以斜杠结尾
# wd = "/your/working/directory"
wd = "/Users/sherlock/Documents/bioinformatics/gwas2hap/database/Oryzadb/"

In [3]:
# 获得只对基因进行注释的文件（不含 CDS 等）
filepath = wd + "Oryza_sativa.geneonly.gff3"
osa_gene_gff3 = pd.read_csv(filepath,header=None,sep="\t")

In [4]:
print(osa_gene_gff3.head(),"\n\n","shape:",osa_gene_gff3.shape)

   0              1     2      3      4  5  6  7  \
0  1  RAP2018-11-26  gene   2983  10815  .  +  .   
1  1  RAP2018-11-26  gene  11218  12435  .  +  .   
2  1  RAP2018-11-26  gene  11372  12284  .  -  .   
3  1  RAP2018-11-26  gene  12721  15685  .  +  .   
4  1  RAP2018-11-26  gene  12808  13978  .  -  .   

                                                   8  
0  ID=gene:Os01g0100100;biotype=protein_coding;de...  
1  ID=gene:Os01g0100200;biotype=protein_coding;de...  
2  ID=gene:Os01g0100300;biotype=protein_coding;de...  
3  ID=gene:Os01g0100400;biotype=protein_coding;de...  
4  ID=gene:Os01g0100466;biotype=protein_coding;de...   

 shape: (37852, 9)


In [5]:
# 我需要的基因注释信息都来自第 8 列，但第 8 列的格式并不规整，比如有的基因有 Name 有的却没有
# 以其中两行为例，可以看出直接用分号分隔无法对齐
print(osa_gene_gff3.iloc[0,8],"\n\n",osa_gene_gff3.iloc[100,8])

ID=gene:Os01g0100100;biotype=protein_coding;description=RabGAP/TBC domain containing protein;gene_id=Os01g0100100;logic_name=rapdb_genes 

 ID=gene:Os01g0111900;Name=SIP34;biotype=protein_coding;description=Glutelin family protein;gene_id=Os01g0111900;logic_name=rapdb_genes


In [6]:
# 第 8 列共有 6 个 Key，分别是：ID=gene，Name，biotype，description，gene_id，logic_name
# 我只需要前 4 个 Key，后面两个 Key 冗余或无关紧要
# 思路是把 8 列里的每个部分拆成 key 列和 value 列，然后查看 key 列里真实的 key 值，把 key 值相符和不相符的分门别类，整理后再合并

In [7]:
# 获得第 8 列的注释信息
gene_dsp = osa_gene_gff3[8].str.split(";",expand=True)
gene_dsp.head()

,0,1,2,3,4,5
0,ID=gene:Os01g0100100,biotype=protein_coding,description=RabGAP/TBC domain containing protein,gene_id=Os01g0100100,logic_name=rapdb_genes,None
1,ID=gene:Os01g0100200,biotype=protein_coding,description=Conserved hypothetical protein,gene_id=Os01g0100200,logic_name=rapdb_genes,None
2,ID=gene:Os01g0100300,biotype=protein_coding,description=Cytochrome P450 domain containing ...,gene_id=Os01g0100300,logic_name=rapdb_genes,None
3,ID=gene:Os01g0100400,biotype=protein_coding,description=Similar to Pectinesterase-like pro...,gene_id=Os01g0100400,logic_name=rapdb_genes,None
4,ID=gene:Os01g0100466,biotype=protein_coding,description=Hypothetical protein,gene_id=Os01g0100466,logic_name=rapdb_genes,None


In [8]:
# 把注释信息里的第一部分 ID=gene 提取出来作为一个 df
# 因为 ID=gene 分隔符是冒号，而其余部分分隔符是等号，因此单独拿出来处理，其余部分则可在下面的循环中完成
geneID = gene_dsp[0].str.split(":",expand=True)
geneID.columns = ['key_geneID','value_geneID']
print(geneID.head(),"\n\n","value_counts:",geneID["key_geneID"].value_counts())

  key_geneID  value_geneID
0    ID=gene  Os01g0100100
1    ID=gene  Os01g0100200
2    ID=gene  Os01g0100300
3    ID=gene  Os01g0100400
4    ID=gene  Os01g0100466 

 value_counts: ID=gene    37852
Name: key_geneID, dtype: int64


In [9]:
# 把注释信息里剩余部分一一提取出来作为 df 
df_list = [geneID]
colnames = ["geneID","biotype","dsp","geneid"]
for i in range(1,len(colnames)):
    df = gene_dsp[i].str.split("=",expand=True)
  
    colname = colnames[i]
#     print(colname)
    colname_key = "key_" + colname
    colname_value = "value_" + colname
    df.columns = [colname_key,colname_value]
#     print(df.head(2))
    print(df[colname_key].value_counts())
    print("***********"*5)
    
    df_list.append(df) 

biotype    28134
Name        9718
Name: key_biotype, dtype: int64
*******************************************************
description    28023
biotype         9718
gene_id          111
Name: key_dsp, dtype: int64
*******************************************************
gene_id        28023
description     9718
logic_name       111
Name: key_geneid, dtype: int64
*******************************************************


In [10]:
# 将所有 df 合并
gene_dsp_con = pd.concat(df_list,axis=1)
print(gene_dsp_con.head(),"\n\n","shape:",gene_dsp_con.shape)

  key_geneID  value_geneID key_biotype   value_biotype      key_dsp  \
0    ID=gene  Os01g0100100     biotype  protein_coding  description   
1    ID=gene  Os01g0100200     biotype  protein_coding  description   
2    ID=gene  Os01g0100300     biotype  protein_coding  description   
3    ID=gene  Os01g0100400     biotype  protein_coding  description   
4    ID=gene  Os01g0100466     biotype  protein_coding  description   

                                   value_dsp key_geneid  value_geneid  
0       RabGAP/TBC domain containing protein    gene_id  Os01g0100100  
1             Conserved hypothetical protein    gene_id  Os01g0100200  
2  Cytochrome P450 domain containing protein    gene_id  Os01g0100300  
3     Similar to Pectinesterase-like protein    gene_id  Os01g0100400  
4                       Hypothetical protein    gene_id  Os01g0100466   

 shape: (37852, 8)


In [11]:
# 把 key 相符的提取出来作为 right，不相符的作为 left
right = gene_dsp_con[gene_dsp_con["key_biotype"] == "biotype"]
left = gene_dsp_con[gene_dsp_con["key_biotype"] == "Name"]

In [12]:
right.head()

,key_geneID,value_geneID,key_biotype,value_biotype,key_dsp,value_dsp,key_geneid,value_geneid
0,ID=gene,Os01g0100100,biotype,protein_coding,description,RabGAP/TBC domain containing protein,gene_id,Os01g0100100
1,ID=gene,Os01g0100200,biotype,protein_coding,description,Conserved hypothetical protein,gene_id,Os01g0100200
2,ID=gene,Os01g0100300,biotype,protein_coding,description,Cytochrome P450 domain containing protein,gene_id,Os01g0100300
3,ID=gene,Os01g0100400,biotype,protein_coding,description,Similar to Pectinesterase-like protein,gene_id,Os01g0100400
4,ID=gene,Os01g0100466,biotype,protein_coding,description,Hypothetical protein,gene_id,Os01g0100466


In [13]:
right.shape

(28134, 8)

In [14]:
left.head()

,key_geneID,value_geneID,key_biotype,value_biotype,key_dsp,value_dsp,key_geneid,value_geneid
8,ID=gene,Os01g0100700,Name,RPS5,biotype,protein_coding,description,Similar to 40S ribosomal protein S5-1
10,ID=gene,Os01g0100900,Name,SPL1,biotype,protein_coding,description,Sphingosine-1-phosphate lyase%2C Disease resis...
16,ID=gene,Os01g0101700,Name,OsDjC1,biotype,protein_coding,description,Similar to chaperone protein dnaJ 20
20,ID=gene,Os01g0102000,Name,OsNPC6,biotype,protein_coding,description,Phosphoesterase family protein
21,ID=gene,Os01g0102300,Name,OsTLP27,biotype,protein_coding,description,Thylakoid lumen protein%2C Photosynthesis and ...


In [15]:
left.shape

(9718, 8)

In [16]:
# right 的 geneID，biotype 和 description（description 里混入了 111 个 geneid，但无伤大雅）都相符
# 只需再增加一列 NAME 赋空值，稍后与 left 合并即可
right_value = right[["value_geneID","value_biotype","value_dsp"]]
right_value.columns = ["GENE","BIOTYPE","DESCRIPTION"]
right_value["NAME"] = np.nan
right_value.head()

/Users/sherlock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,GENE,BIOTYPE,DESCRIPTION,NAME
0,Os01g0100100,protein_coding,RabGAP/TBC domain containing protein,NaN
1,Os01g0100200,protein_coding,Conserved hypothetical protein,NaN
2,Os01g0100300,protein_coding,Cytochrome P450 domain containing protein,NaN
3,Os01g0100400,protein_coding,Similar to Pectinesterase-like protein,NaN
4,Os01g0100466,protein_coding,Hypothetical protein,NaN


In [17]:
# left 的 biotype 列其实是 name，description 列其实是 biotype，这些错位之处通过更改列名的方式即可纠正
left_value = left[["value_geneID","value_dsp","value_geneid","value_biotype"]]
left_value.columns = ["GENE","BIOTYPE","DESCRIPTION","NAME"]
left_value.head()

,GENE,BIOTYPE,DESCRIPTION,NAME
8,Os01g0100700,protein_coding,Similar to 40S ribosomal protein S5-1,RPS5
10,Os01g0100900,protein_coding,Sphingosine-1-phosphate lyase%2C Disease resis...,SPL1
16,Os01g0101700,protein_coding,Similar to chaperone protein dnaJ 20,OsDjC1
20,Os01g0102000,protein_coding,Phosphoesterase family protein,OsNPC6
21,Os01g0102300,protein_coding,Thylakoid lumen protein%2C Photosynthesis and ...,OsTLP27


In [18]:
# 将 right 和 left 合并即完整的基因注释
osa_gene_anno = pd.concat([right_value,left_value])
osa_gene_anno = osa_gene_anno.sort_values("GENE")
osa_gene_anno = osa_gene_anno[["GENE","NAME","BIOTYPE","DESCRIPTION"]]
osa_gene_anno.head()

,GENE,NAME,BIOTYPE,DESCRIPTION
0,Os01g0100100,NaN,protein_coding,RabGAP/TBC domain containing protein
1,Os01g0100200,NaN,protein_coding,Conserved hypothetical protein
2,Os01g0100300,NaN,protein_coding,Cytochrome P450 domain containing protein
3,Os01g0100400,NaN,protein_coding,Similar to Pectinesterase-like protein
4,Os01g0100466,NaN,protein_coding,Hypothetical protein


In [19]:
# 随机查看其中 5 行
osa_gene_anno.sample(5)

,GENE,NAME,BIOTYPE,DESCRIPTION
14637,Os02g0633700,OsPP2C23,protein_coding,Similar to T-cell activation protein phosphata...
262,Os01g0129200,SL1,protein_coding,C2H2 zinc-finger transcription factor%2C Flora...
11640,Os12g0595200,Os_F0776,protein_coding,Similar to F-box domain containing protein
25233,Os05g0328000,NaN,protein_coding,Hypothetical protein
11436,Os12g0562600,NaN,protein_coding,Similar to Protein kinase


In [20]:
# 保存到本地
savepath = wd + "Oryza_sativa_gene_anno.gff3"
osa_gene_anno.to_csv(savepath,index=None,sep="\t")